In [1]:
from __init__ import *
from snmcseq_utils import cd
from snmcseq_utils import create_logger
from CEMBA_update_mysql import connect_sql


In [2]:
log = create_logger()
log.info("Hello")

04/18/2018 12:27:03 PM Hello


In [37]:
ens = 'Ens0'
cluster_type = 'mCH_npc50_k30_merged_subsampled'
# annotation_type = 'annotation' + cluster_type[len('cluster'):]
# engine = connect_sql('CEMBA')

# cluster annotation
# sql = """SELECT {}, {} FROM {}""".format(cluster_type, annotation_type, ens)
# df_info = pd.read_sql(sql, engine)
# df_info = df_info.sort_values(cluster_type).drop_duplicates().set_index(cluster_type)

ens_path = os.path.join(PATH_ENSEMBLES, ens)
n_clusters = len(glob.glob(
    os.path.join(ens_path, 'allc_merged/{}/allc_merged_mCG_{}_*_{}.tsv'.format(cluster_type, cluster_type, ens))))

# dmr results
input_f = os.path.join(ens_path, 'dmr/dmr_allc_merged_mCG_{}_rms_results_collapsed.tsv'.format(cluster_type))
df = pd.read_table(input_f, index_col=['#chr', 'start', 'end'], dtype={'#chr': object})

/cndd/fangming/venvs/venv_dobro_snmcseq/lib/python3.5/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [17]:
key = 'hypomethylated_samples'
num_dms = 3 
res = df.loc[(~df[key].isnull()) & (df['number_of_dms'] >= num_dms), :].copy()
res['name'] = np.arange(res.shape[0]) 
print(res.shape)

# with cd(os.path.join(PATH_ENSEMBLES, ens, 'dmr')):
#     res[['name', 'number_of_dms']].to_csv('dmr_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_rms_results_collapsed_{}dms.tsv.DMR.bed'.format(num_dms),
#                                      sep='\t', na_rep='NA', header=False, index=True)



(733635, 21)


17

In [18]:
def dmr_counts(ens, n_clusters, cluster_type, df, num_dms=3, keys=['hypomethylated_samples', 'hypermethylated_samples']):
    """
    cluster_type examples:
    cluster_type = 'cluster_mCHmCG_lv_npc50_k30'
    
    """
    res_out = []
    for key in keys:
        res_key = []
        res = df.loc[(~df[key].isnull()) & (df['number_of_dms'] >= num_dms), key].apply(lambda x: x.split(','))
        for i in range(n_clusters):
            res_cluster = res.loc[
                res.apply(lambda x: ('merged_mCG_{}_{}_{}'.format(cluster_type, i+1, ens) in x))]
            res_key.append({cluster_type: i+1, 'n_dmr_{}'.format(key[:4]): res_cluster.shape[0]})
        res_key = pd.DataFrame(res_key).set_index(cluster_type)
        res_out.append(res_key)
    return pd.concat(res_out, axis=1)

In [22]:
# df_ndmr = dmr_counts(ens, n_clusters, cluster_type, df, num_dms=3)
# df_ndmr = pd.merge(df_info, df_ndmr, left_index=True, right_index=True)

df.head()

number_of_dms  \
#chr start end                    
1    13417 13537              4   
     13823 13828              2   
     54489 54489              1   
     56298 56305              2   
     57691 57691              1   

                                            hypermethylated_samples  \
#chr start end                                                        
1    13417 13537  merged_mCG_mCH_npc50_k30_merged_1_Ens0_mergest...   
     13823 13828                                                NaN   
     54489 54489                                                NaN   
     56298 56305                                                NaN   
     57691 57691                                                NaN   

                                             hypomethylated_samples  \
#chr start end                                                        
1    13417 13537  merged_mCG_mCH_npc50_k30_merged_10_Ens0_merges...   
     13823 13828  merged_mCG_mCH_npc50_k30_merged_10_Ens0_merges...   
     54489 54489                                                NaN   
     56298 56305  merged_mCG_mCH_npc50_k30_merged_15_Ens0_merges...   
     57691 57691  merged_mCG_mCH_npc50_k30_merged_1_Ens0_mergest...   

                  methylation_level_merged_mCG_mCH_npc50_k30_merged_10_Ens0_mergestrands  \
#chr start end                                                                             
1    13417 13537                                           0.650000                        
     13823 13828                                           0.263158                        
     54489 54489                                           0.538462                        
     56298 56305                                           0.840000                        
     57691 57691                                           0.750000                        

                  methylation_level_merged_mCG_mCH_npc50_k30_merged_11_Ens0_mergestrands  \
#chr start end                                                                             
1    13417 13537                                           0.482072                        
     13823 13828                                           0.872727                        
     54489 54489                                           0.714286                        
     56298 56305                                           0.575758                        
     57691 57691                                                NaN                        

                  methylation_level_merged_mCG_mCH_npc50_k30_merged_14_Ens0_mergestrands  \
#chr start end                                                                             
1    13417 13537                                           0.691824                        
     13823 13828                                           0.863636                        
     54489 54489                                           0.500000                        
     56298 56305                                           0.666667                        
     57691 57691                                           0.500000                        

                  methylation_level_merged_mCG_mCH_npc50_k30_merged_15_Ens0_mergestrands  \
#chr start end                                                                             
1    13417 13537                                           0.944444                        
     13823 13828                                           0.476190                        
     54489 54489                                           0.428571                        
     56298 56305                                           0.210526                        
     57691 57691                                           0.666667                        

                  methylation_level_merged_mCG_mCH_npc50_k30_merged_1_Ens0_mergestrands  \
#chr start end                                                                            
1    13417 135

In [41]:
print(df.shape)
df_hypo = df.loc[(df['number_of_dms']>=3) & (~df['hypomethylated_samples'].isnull()), 'hypomethylated_samples'].apply(lambda x: x.split(','))
df_hyper = df.loc[(df['number_of_dms']>=3) & (~df['hypermethylated_samples'].isnull()), 'hypermethylated_samples'].apply(lambda x: x.split(','))

print(df_hypo.shape)
print(df_hyper.shape)

(392153, 20)
(60718,)
(44613,)


In [42]:
df_hypo.head()

#chr  start    end    
1     3391591  3391804      [merged_mCG_cluster_mCHmCG_lv_npc50_k30_1_Ens1]
      3498321  3498399    [merged_mCG_cluster_mCHmCG_lv_npc50_k30_5_Ens1...
      3637646  3637688      [merged_mCG_cluster_mCHmCG_lv_npc50_k30_4_Ens1]
      4471128  4471240      [merged_mCG_cluster_mCHmCG_lv_npc50_k30_2_Ens1]
      4688593  4688647    [merged_mCG_cluster_mCHmCG_lv_npc50_k30_4_Ens1...
Name: hypomethylated_samples, dtype: object

In [45]:
if not os.path.isdir('/scratch/dmr'):
    os.makedirs('/scratch/dmr')
    logging.info('Created directory: /scratch/dmr')
    
for i in range(n_clusters):
    df_hypo_cluster = df_hypo.loc[
        df_hypo.apply(lambda x: ('merged_mCG_{}_{}_{}'.format(cluster_type, i+1, ens) in x))]
    output = os.path.join('/scratch/dmr', 'dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst{}.bed'.format(i+1))
    df_hypo_cluster.to_csv(output, sep='\t', header=False, index=True, na_rep='NA')
    logging.info("Saved to {}".format(output))
    logging.info(df_hypo_cluster.shape)

03/04/2018 08:24:43 PM Created directory: /scratch/dmr
03/04/2018 08:24:43 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst1.bed
03/04/2018 08:24:43 PM (16977,)
03/04/2018 08:24:44 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst2.bed
03/04/2018 08:24:44 PM (19580,)
03/04/2018 08:24:45 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst3.bed
03/04/2018 08:24:45 PM (14284,)
03/04/2018 08:24:45 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst4.bed
03/04/2018 08:24:45 PM (6308,)
03/04/2018 08:24:46 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst5.bed
03/04/2018 08:24:46 PM (11274,)
03/04/2018 08:24:46 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst6.bed
03/04/2018 08:24:46 PM (7308,)
03/04/2018 08:24:47 PM Saved to /scratch/dmr/dmr_hypo_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_clst7.bed

In [21]:
with cd(os.path.join(PATH_ENSEMBLES, ens, 'dmr')):
    f = 'dmr_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_rms_results_collapsed.tsv.DMR.bed'
    df_bed = pd.read_table(f, header=None, names=['chr', 'start', 'end', 'num_dms'])
print(df_bed.shape)
df_bed.head()

(1162175, 4)


,chr,start,end,num_dms
0,chr1,3013472,3013473,1
1,chr1,3023074,3023075,1
2,chr1,3027397,3027398,1
3,chr1,3034471,3034472,1
4,chr1,3044312,3044403,3


In [23]:
df_out = df_bed[df_bed['num_dms']>=20]
with cd(os.path.join(PATH_ENSEMBLES, ens, 'dmr')):
    df_out.to_csv('dmr_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_rms_results_collapsed_20dms.tsv.DMR.bed', 
                 sep='\t', na_rep='NA', header=False, index=False)

## each cluster hypo

In [38]:
df.head()

number_of_dms  \
#chr start end                    
1    13417 13537              4   
     13823 13828              2   
     56298 56305              2   
     78894 78894              1   
     79417 79417              1   

                                            hypermethylated_samples  \
#chr start end                                                        
1    13417 13537  merged_mCG_mCH_npc50_k30_merged_subsampled_37_...   
     13823 13828                                                NaN   
     56298 56305                                                NaN   
     78894 78894                                                NaN   
     79417 79417                                                NaN   

                                             hypomethylated_samples  \
#chr start end                                                        
1    13417 13537  merged_mCG_mCH_npc50_k30_merged_subsampled_10_...   
     13823 13828  merged_mCG_mCH_npc50_k30_merged_subsampled_10_...   
     56298 56305  merged_mCG_mCH_npc50_k30_merged_subsampled_15_...   
     78894 78894                                                NaN   
     79417 79417  merged_mCG_mCH_npc50_k30_merged_subsampled_39_...   

                  methylation_level_merged_mCG_mCH_npc50_k30_merged_subsampled_10_Ens0_mergestrands  \
#chr start end                                                                                        
1    13417 13537                                           0.632911                                   
     13823 13828                                           0.235294                                   
     56298 56305                                           0.800000                                   
     78894 78894                                           1.000000                                   
     79417 79417                                           0.833333                                   

                  methylation_level_merged_mCG_mCH_npc50_k30_merged_subsampled_11_Ens0_mergestrands  \
#chr start end                                                                                        
1    13417 13537                                           0.488038                                   
     13823 13828                                           0.872727                                   
     56298 56305                                           0.580645                                   
     78894 78894                                                NaN                                   
     79417 79417                                           0.000000                                   

                  methylation_level_merged_mCG_mCH_npc50_k30_merged_subsampled_14_Ens0_mergestrands  \
#chr start end                                                                                        
1    13417 13537                                           0.691824                                   
     13823 13828                                           0.863636                                   
     56298 56305                                           0.666667                                   
     78894 78894                                                NaN                                   
     79417 79417                                           0.000000                                   

                  methylation_level_merged_mCG_mCH_npc50_k30_merged_subsampled_15_Ens0_mergestrands  \
#chr start end                                                                                        
1    13417 13537                                           0.944444                                   
     13823 13828                                           0.476190                                   
     56298 56305                                           0.210526                                   
     78894 78894                                           1.000000                            

In [39]:
df_hypo = df[(~df['hypomethylated_samples'].isnull()) & (df['number_of_dms']>=3)]

df_new = df_hypo.copy()
df_new['hypomethylated_samples'] = df_hypo['hypomethylated_samples'].apply(lambda x: x.split(','))

# unique
df_new['num_clusters'] = df_new['hypomethylated_samples'].apply(lambda x: len(x))
df_new = df_new[df_new['num_clusters']==1]
df_new['hypomethylated_samples'] = df_hypo['hypomethylated_samples'] 

print(df_new.shape)
df_new.head()

(64491, 21)


number_of_dms  \
#chr start  end                     
1    714862 715510             16   
     748228 748544              3   
     792049 792185              5   
     805500 805942             10   
     817520 817724              3   

                                              hypermethylated_samples  \
#chr start  end                                                         
1    714862 715510  merged_mCG_mCH_npc50_k30_merged_subsampled_33_...   
     748228 748544  merged_mCG_mCH_npc50_k30_merged_subsampled_34_...   
     792049 792185                                                NaN   
     805500 805942  merged_mCG_mCH_npc50_k30_merged_subsampled_11_...   
     817520 817724                                                NaN   

                                               hypomethylated_samples  \
#chr start  end                                                         
1    714862 715510  merged_mCG_mCH_npc50_k30_merged_subsampled_37_...   
     748228 748544  merged_mCG_mCH_npc50_k30_merged_subsampled_4_E...   
     792049 792185  merged_mCG_mCH_npc50_k30_merged_subsampled_14_...   
     805500 805942  merged_mCG_mCH_npc50_k30_merged_subsampled_10_...   
     817520 817724  merged_mCG_mCH_npc50_k30_merged_subsampled_14_...   

                    methylation_level_merged_mCG_mCH_npc50_k30_merged_subsampled_10_Ens0_mergestrands  \
#chr start  end                                                                                         
1    714862 715510                                           0.520661                                   
     748228 748544                                           0.547619                                   
     792049 792185                                           0.790698                                   
     805500 805942                                           0.042553                                   
     817520 817724                                           0.950000                                   

                    methylation_level_merged_mCG_mCH_npc50_k30_merged_subsampled_11_Ens0_mergestrands  \
#chr start  end                                                                                         
1    714862 715510                                           0.605166                                   
     748228 748544                                           0.784615                                   
     792049 792185                                           0.818182                                   
     805500 805942                                           0.335749                                   
     817520 817724                                           0.785714                                   

                    methylation_level_merged_mCG_mCH_npc50_k30_merged_subsampled_14_Ens0_mergestrands  \
#chr start  end                                                                                         
1    714862 715510                                           0.575758                                   
     748228 748544                                           0.613636                                   
     792049 792185                                           0.422222                                   
     805500 805942                                           0.212346                                   
     817520 817724                                           0.537037                                   

                    methylation_level_merged_mCG_mCH_npc50_k30_merged_subsampled_15_Ens0_mergestrands  \
#chr start  end                                                                                         
1    714862 715510                                           0.388350                                   
     748228 748544                                           0.285714                                   
     792049 792185                                           0.877778                                   
   

In [40]:
df_new.groupby('hypomethylated_samples').count()['number_of_dms']

hypomethylated_samples
merged_mCG_mCH_npc50_k30_merged_subsampled_10_Ens0_mergestrands     1772
merged_mCG_mCH_npc50_k30_merged_subsampled_11_Ens0_mergestrands     8389
merged_mCG_mCH_npc50_k30_merged_subsampled_14_Ens0_mergestrands    28620
merged_mCG_mCH_npc50_k30_merged_subsampled_15_Ens0_mergestrands      685
merged_mCG_mCH_npc50_k30_merged_subsampled_1_Ens0_mergestrands      1353
merged_mCG_mCH_npc50_k30_merged_subsampled_2_Ens0_mergestrands      2267
merged_mCG_mCH_npc50_k30_merged_subsampled_33_Ens0_mergestrands     1145
merged_mCG_mCH_npc50_k30_merged_subsampled_34_Ens0_mergestrands      811
merged_mCG_mCH_npc50_k30_merged_subsampled_35_Ens0_mergestrands     1639
merged_mCG_mCH_npc50_k30_merged_subsampled_36_Ens0_mergestrands     1210
merged_mCG_mCH_npc50_k30_merged_subsampled_37_Ens0_mergestrands     1441
merged_mCG_mCH_npc50_k30_merged_subsampled_38_Ens0_mergestrands      338
merged_mCG_mCH_npc50_k30_merged_subsampled_39_Ens0_mergestrands     1846
merged_mCG_mCH_npc50_k30_mer

In [41]:
cluster_files = glob.glob(
    os.path.join(ens_path, 'allc_merged/{}/allc_merged_mCG_{}_*_{}.tsv'.format(cluster_type, cluster_type, ens)))

clusters = sorted([int(os.path.basename(cluster_f).split('_')[-2]) for cluster_f in cluster_files])

dict_outs = dict()
for i in clusters:
    dict_outs[i] = os.path.join(PATH_ENSEMBLES, ens, 'dmr', cluster_type, '{}_hypo_unique.bed'.format(i))

In [42]:
dict_outfs = dict()
# for i in clusters:
#     dict_outfs[i] = open(dict_outs[i], 'w')
# dict_outfs
for i in clusters:
    dict_outfs[i] = [] 
dict_outfs


for i, (idx, row) in enumerate(df_new.reset_index().iterrows()):
    if i % 10000 == 0:
        print("Progress {}".format(i+1))
        
    if isinstance(row['hypomethylated_samples'], str):
        item = row['hypomethylated_samples']
        cluster = int(item.split('_')[-3])
        dict_outfs[cluster].append({'chr': row['#chr'],
                                   'start': row['start'],
                                   'end': row['end'],
                                   'number_of_dms': row['number_of_dms']})
        
    elif isinstance(row['hypomethylated_samples'], list):
        for item in row['hypomethylated_samples']:
            cluster = int(item.split('_')[-3])
            dict_outfs[cluster].append({'chr': row['#chr'],
                                       'start': row['start'],
                                       'end': row['end'],
                                       'number_of_dms': row['number_of_dms']})


for i in clusters:
    dict_outfs[i] = pd.DataFrame(dict_outfs[i])[['chr', 'start', 'end', 'number_of_dms']]

Progress 1
Progress 10001
Progress 20001
Progress 30001
Progress 40001
Progress 50001
Progress 60001


In [43]:
for i in clusters:
    dict_outfs[i].to_csv(dict_outs[i], 
                         sep='\t', na_rep='NA', header=False, index=False)